<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/LLM/training/examples_fine_tune_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pub.towardsai.net/fine-tuning-llms-from-zero-to-hero-with-python-ollama-52258966bb6d

In [9]:
# Downgrade protobuf to a compatible version - otherwise save_pretrained_gguf fails on google colab
!pip install "protobuf>=3.19.0,<4.0.0" --quiet
# also for google colab
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [10]:

!pip install unsloth --quiet
# !pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes transformers datasets --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 17.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os
os.environ['WANDB_API_KEY']=userdata.get('WANDB_API_KEY')


Load the basemodel in unsloth

In [11]:
from unsloth import FastLanguageModel

model_name = "unsloth/phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Adjust based on your data length
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Add LoRA Adapter to the layers for efficient training

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank - higher = more parameters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


The training dataset that we read in looks like:

[
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
]

In [4]:
!wget --no-cache https://raw.githubusercontent.com/ebamberg/research-projects-ml/refs/heads/main/LLM/training/logfile_data_synthetic_train.json -O training_data.json
!wget --no-cache https://raw.githubusercontent.com/ebamberg/research-projects-ml/refs/heads/main/LLM/training/logfile_data_synthetic_eval.json -O eval_data.json

--2025-08-25 14:31:17--  https://raw.githubusercontent.com/ebamberg/research-projects-ml/refs/heads/main/LLM/training/logfile_data_synthetic_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49528 (48K) [text/plain]
Saving to: ‘training_data.json’

training_data.json  100%[===================>]  48.37K  --.-KB/s    in 0.009s  

2025-08-25 14:31:17 (5.55 MB/s) - ‘training_data.json’ saved [49528/49528]

--2025-08-25 14:31:17--  https://raw.githubusercontent.com/ebamberg/research-projects-ml/refs/heads/main/LLM/training/logfile_data_synthetic_eval.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.

In [13]:
import json
from datasets import Dataset

with open("training_data.json", "r") as f:
    data = json.load(f)
# Format for training
def format_chat_template(item):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": item['input']},
            {"role": "assistant", "content": item['output']}
        ],
        tokenize=False,
        add_generation_prompt=False
    )
# Create the training dataset
formatted_data = [{"text": format_chat_template(item)} for item in data]
dataset = Dataset.from_list(formatted_data)
# Check what it looks like
print("Sample training example:")
print(formatted_data[0]["text"])

Sample training example:
<|user|>
classify the following log event sequence as normal or suspicious
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:22:26,User authentication successful,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:23:26,User logout successful,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:28:26,Database query executed,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:32:26,Connection established,DEBUG
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:38:26,Database query executed,INFO
192.168.1.20,sess_391494,alex.martin,2025-07-21 14:52:26,User logout successful,INFO<|end|>
<|assistant|>
normal<|end|>
<|endoftext|>


We are ready to go, so start the training !

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

training_steps=60

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=1, # Reduced batch size
        gradient_accumulation_steps=8, # Increased accumulation steps to maintain similar effective batch size
        warmup_steps=5,
        max_steps=training_steps,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",  # Use "adamw_torch" if you get optimizer errors
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=30,
    ),
)
# Start training! 🚀
trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/70 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 70 | Num Epochs = 7 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erik-bamberg (erik-bamberg-self-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.767500
2,0.730500
3,0.719300
4,0.767200
5,0.648000
6,0.678700
7,0.620000
8,0.514300
9,0.506300
10,0.485100


Evaluate our model

In [8]:
test = "classify the following log event sequence as normal or suspicious\n91.189.89.199,sess_752130,root,2025-07-27 14:22:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:24:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:26:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:28:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:30:26,Authentication failed for user root,ERROR\n91.189.89.199,sess_752130,root,2025-07-27 14:32:26,Authentication failed for user root,ERROR\n91.189.89.199,sess_752130,root,2025-07-27 14:34:26,Authentication failed for user root,ERROR\n185.220.101.42,sess_752130,root,2025-07-27 14:36:26,Authentication failed for user root,ERROR\n91.189.89.199,sess_752130,root,2025-07-27 14:42:26,Brute force attack identified,ERROR"
print(test)

classify the following log event sequence as normal or suspicious
91.189.89.199,sess_752130,root,2025-07-27 14:22:26,Authentication failed for user root,ERROR
185.220.101.42,sess_752130,root,2025-07-27 14:24:26,Authentication failed for user root,ERROR
185.220.101.42,sess_752130,root,2025-07-27 14:26:26,Authentication failed for user root,ERROR
185.220.101.42,sess_752130,root,2025-07-27 14:28:26,Authentication failed for user root,ERROR
185.220.101.42,sess_752130,root,2025-07-27 14:30:26,Authentication failed for user root,ERROR
91.189.89.199,sess_752130,root,2025-07-27 14:32:26,Authentication failed for user root,ERROR
91.189.89.199,sess_752130,root,2025-07-27 14:34:26,Authentication failed for user root,ERROR
185.220.101.42,sess_752130,root,2025-07-27 14:36:26,Authentication failed for user root,ERROR
91.189.89.199,sess_752130,root,2025-07-27 14:42:26,Brute force attack identified,ERROR


In [ ]:
# Switch to inference mode
FastLanguageModel.for_inference(model)


# Format input using chat template
messages = [{"role": "user", "content": test}]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")
# Generate response
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        use_cache=True,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
# Extract just the new generated text (not the full conversation)
response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print("Model output:")
print(response.strip())


SAVE our fine-tuned model in the GGUF format which is compatible to Ollama.

In [ ]:
# save_pretrained_gguf is a unsloth function. this is ot available on standard hugging face models
model.save_pretrained_gguf(
    "fine_tuned_model",
    tokenizer,
    quantization_method="q4_k_m"  # Good balance of size/quality
)

In [ ]:
%%writefile ./fine_tuned_model/Modelfile.py
FROM ./unsloth.Q4_K_M.gguf
PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER stop ["<|endoftext|>"]
TEMPLATE "{{ .Prompt }}"
SYSTEM "You are a specialized devop trained to analyze log file."

In [ ]:
!zip -r /content/fine_tuned_model_log_file_analyser.zip /content/fine_tuned_model/

from google.colab import files
files.download("/content/fine_tuned_model_log_file_analyser.zip")

you can use ollama to create a model from the modelfile:

ollama create \<modelname\> -f Modelfile

In [ ]:
!pip install ollama --quiet


In [ ]:
!cd /content/fine_tuned_model/
get_ipython().system_raw("ollama create log_file_analyser -f Modelfile")